# ETL from Source to TemplateDB

This ETL area serves multiple purposes:

1. Reset the data items in TemplateDB if existing
2. Load data into TemplateDB from source systems

It's supposed to run on the TemplateDB database, but some checks can be done on the master database of SQL Server.

If possible, it should run without a staging area.

In [ ]:
USE master
GO

DECLARE @errorMSG varchar(300) = ''

IF DB_ID('JKU-Student-System') IS NULL
   SET @errorMSG = @errorMSG + 'JKU-Student-System missing' + CHAR(13)
IF DB_ID('MedUniWien-Student-System') IS NULL
   SET @errorMSG = @errorMSG +  'MedUniWien-Student-System missing' + CHAR(13)
IF DB_ID('TUGraz-Student-System') IS NULL
   SET @errorMSG = @errorMSG +  'TUGraz-Student-System missing' + CHAR(13)
IF DB_ID('TemplateDB') IS NULL
   SET @errorMSG = @errorMSG +  'TemplateDB missing' + CHAR(13)

if (@errorMSG is null or @errorMSG = '')
    print 'All databases seem good to go'
else 
    print @errorMSG

## Empty and reset of TemplateDB

Drops everything from the TemplateDB tables and resets identity colums.

In [ ]:
USE TemplateDB;

DELETE FROM [stage].[Stage_TemplateDB];

DELETE FROM dbo.Registration;
DELETE FROM dbo.Student;
DELETE FROM dbo.Study;
DELETE FROM dbo.Exam;
DELETE FROM dbo.Course;
DELETE FROM dbo.University;

DBCC CHECKIDENT ('[stage].[Stage_TemplateDB]', RESEED, 0);

DBCC CHECKIDENT ('[dbo].[Student]', RESEED, 0);
DBCC CHECKIDENT ('[dbo].[Study]', RESEED, 0);
DBCC CHECKIDENT ('[dbo].[Exam]', RESEED, 0);
DBCC CHECKIDENT ('[dbo].[Course]', RESEED, 0);
DBCC CHECKIDENT ('[dbo].[University]', RESEED, 0);

## Load data into TemplateDB Stage

### Load from JKU to stage

In [ ]:
USE TemplateDB;

DECLARE @uniName VARCHAR(50) = 'JKU';

INSERT INTO [stage].[Stage_TemplateDB]
([name_nk], [faculty], [study_program], [gender], [country], [city], [grade], [year], [semester], [month], [institute], [course_name], [ects])
SELECT @uniName, sp.Faculty, sp.Name, (Case When gender = 0 THEN 'Female' 
    when gender = 1 THEN 'Male'
    Else 'Non-Binary' END ) as Gender, sa.country, sa.city,
    CAST(reg.Grade AS INT) as Grade, RIGHT(reg.Semester, 4), reg.Semester, '', 
    p.Institute, c.course_name, c.ECTS
FROM [JKU-Student-System].[dbo].[Student] s
JOIN [JKU-Student-System].[dbo].StudentAddress sa ON s.mail_address = sa.student_address_id
JOIN [JKU-Student-System].[dbo].Registration reg ON s.matr_nr = reg.student_id
JOIN [JKU-Student-System].[dbo].Course c ON reg.course_number = c.course_number
JOIN [JKU-Student-System].dbo.StudyProgram sp ON sp.study_id = reg.study_id
JOIN [JKU-Student-System].[dbo].[Teaching] t ON t.course_number = c.course_number
JOIN [JKU-Student-System].[dbo].[Professor] p ON p.professor_id = t.professor_id
WHERE reg.grade IS NOT NULL AND (t.Ordering = 1 OR t.Ordering IS NULL)


### Load from TUGraz to stage

In [ ]:
USE TemplateDB;

DECLARE @uniName VARCHAR(50) = 'TUGraz';

INSERT INTO [stage].[Stage_TemplateDB]
([name_nk], [faculty], [study_program], [gender], [country], [city], [grade], [year], [semester], [month], [institute], [course_name], [ects])
SELECT @uniName, f.Name as 'Faculty Name', sf.Name as 'Study Name', 
    (Case When gender = 1 THEN 'Female' 
    when gender = 0 THEN 'Male'
    Else 'Non-Binary' END ) as Gender,
    (SELECT nicename FROM [TemplateDB].[stage].[Stage_CountryCodeMap] WHERE iso = (RIGHT(full_address, LEN(full_address) -1 - charindex(',', full_address, charindex(',', full_address, charindex(',', full_address, 1) +1) +1)))),
    SUBSTRING(full_address, charindex(',', full_address, charindex(',', full_address, 1)+1)+2, charindex(',', full_address, charindex(',', full_address, charindex(',', full_address, 1) +1) +1)-charindex(',', full_address, charindex(',', full_address, 1)+1)-2),
    er.Grade, YEAR(e.[Date]) as 'Year',
    (CASE WHEN MONTH(e.[Date]) <= 2 THEN 'WS' + CONVERT(varchar, YEAR(e.[Date] - 1))
        WHEN MONTH(e.[Date]) <= 9 THEN 'SS' + CONVERT(varchar, YEAR(e.[Date]))
        ELSE 'WS' + CONVERT(varchar, YEAR(e.[Date])) END) as 'Semester', 
    DateName( month , DateAdd( month , MONTH(e.[Date]) , -1 )) as 'Month',
    c.Institute, c.Name, c.ECTS

FROM [TUGraz-Student-System].[dbo].Student s
JOIN [TUGraz-Student-System].dbo.StudyField sf ON s.field_of_study = sf.study_id
JOIN [TUGraz-Student-System].[dbo].[Faculty] f ON sf.faculty_id = f.faculty_id
JOIN [TUGraz-Student-System].dbo.CourseRegistration cr ON cr.student_id = s.student_id
JOIN [TUGraz-Student-System].dbo.Course c ON c.course_id = cr.course_id
JOIN [TUGraz-Student-System].dbo.ExamRegistration er ON er.student_id = s.student_id
JOIN (SELECT dateadd(S, [time_from], '1970-01-01') as 'Date', exam_id, course_id FROM [TUGraz-Student-System].[dbo].[Exam]) e ON e.course_id = c.course_id AND e.exam_id = er.exam_id

WHERE er.Grade IS NOT NULL

### Load MedUniWien to stage

In [ ]:
USE TemplateDB;

DECLARE @uniName VARCHAR(50) = 'MedUniWien';

INSERT INTO [stage].[Stage_TemplateDB]
([name_nk], [faculty], [study_program], [gender], [country], [city], [grade], [year], [semester], [month], [institute], [course_name], [ects])
SELECT @uniName, st.Fakultaet, st.Studium_Name,
    (Case When geschlecht = 'Weiblich' THEN 'Female' 
    when geschlecht = 'Männlich' THEN 'Male'
    Else 'Non-Binary' END ) as Gender,
    (SELECT nicename FROM [TemplateDB].[stage].Stage_CountryCodeMap WHERE iso = Land), Stadt,
     pr.Note,
    CONVERT(int, LEFT(Datum, 4)) as 'Year', 
    (Case When CONVERT(int, SUBSTRING(DATUM, 5, 2)) <= 2 THEN 'WS' + CONVERT(varchar, (CONVERT(int, LEFT(Datum, 4)) - 1)) 
        when CONVERT(int, SUBSTRING(DATUM, 5, 2)) <= 9 THEN 'SS' + LEFT(Datum, 4)
        Else 'WS' + LEFT(Datum, 4) END ) as 'Semester',
    DateName( month , DateAdd( month , CONVERT(int, SUBSTRING(DATUM, 5, 2)) , -1 ) ) as 'Month',
    lva.Institut, lva.Name, lva.ECTS

FROM [MedUniWien-Student-System].[dbo].Studierende s
JOIN [MedUniWien-Student-System].[dbo].Semester sem ON s.aktuelles_semester = sem.semester_id
JOIN [MedUniWien-Student-System].[dbo].Studium st ON st.Studium_ID = sem.Studium_ID
JOIN [MedUniWien-Student-System].[dbo].Lehrveranstaltung lva ON lva.semester_id = sem.semester_id
JOIN [MedUniWien-Student-System].[dbo].Pruefung p ON p.LVA_ID = lva.LVA_ID
JOIN [MedUniWien-Student-System].[dbo].PruefungRegistrierung pr ON pr.student_id = s.student_id AND pr.pruefung_id = p.pruefung_id

WHERE pr.Note IS NOT NULL


## Load from stage to TemplateDB tables

#### Load into University table and load ID to stage

In [ ]:
USE TemplateDB;

INSERT INTO dbo.University
(name_nk)
SELECT DISTINCT name_nk
FROM [stage].[Stage_TemplateDB];

UPDATE std
SET std.dwh_university_id = u.university_id
FROM [stage].[Stage_TemplateDB] std
JOIN dbo.University u ON std.name_nk = u.name_nk;


#### Load tables except for registration and load IDs to stage

In [ ]:
USE TemplateDB;

INSERT INTO dbo.Course
(institute, course_name, ects, source_id)
SELECT DISTINCT institute, course_name, ects, dwh_university_id
FROM [stage].[Stage_TemplateDB]

UPDATE std
SET std.dwh_course_sk = c.course_sk
FROM [stage].[Stage_TemplateDB] std
JOIN [dbo].[Course] c ON std.institute = c.institute 
    AND std.course_name = c.course_name AND std.ects = c.ects
    AND std.dwh_university_id = c.source_id;

INSERT INTO dbo.Exam
([grade], [year], [semester], [month], source_id)
SELECT DISTINCT grade, [year], semester, [month], dwh_university_id
FROM [stage].[Stage_TemplateDB];

UPDATE std
SET std.dwh_exam_sk = e.exam_sk
FROM [stage].[Stage_TemplateDB] std
JOIN [dbo].[Exam] e ON std.grade = e.grade AND std.[year] = e.[year] 
    AND std.semester = e.semester AND std.[month] = e.[month]
    AND std.dwh_university_id = e.source_id;

INSERT INTO dbo.Student
([gender], country, city, source_id)
SELECT DISTINCT gender, country, city, dwh_university_id
FROM [stage].[Stage_TemplateDB];

UPDATE std
SET std.dwh_student_sk = s.student_sk
FROM [stage].[Stage_TemplateDB] std
JOIN [dbo].[Student] s ON std.gender = s.gender
    AND std.country = s.country AND std.city = s.city
    AND std.dwh_university_id = s.source_id;

INSERT INTO dbo.Study
(faculty, study_program, source_id)
SELECT DISTINCT faculty, study_program, dwh_university_id
FROM [stage].[Stage_TemplateDB];

UPDATE std
SET std.dwh_study_sk = s.study_sk
FROM [stage].[Stage_TemplateDB] std
JOIN [dbo].[Study] s ON std.faculty = s.faculty AND std.study_program = s.study_program
    AND std.dwh_university_id = s.source_id;

#### Load registration table with data

In [ ]:
USE TemplateDB

INSERT INTO [dbo].[Registration]
(course_sk, exam_sk, student_sk, study_sk)
SELECT DISTINCT dwh_course_sk, dwh_exam_sk, dwh_student_sk, dwh_study_sk
FROM [stage].[Stage_TemplateDB];